In [ ]:
'''
Tutorial of Lagrangian-relaxation with CPLEX
https://dataplatform.cloud.ibm.com/exchange/public/entry/view/133dfc4cd1480bbe4eaa78d3f6d12e8e?context=cpdaas
http://ibmdecisionoptimization.github.io/docplex-doc/mp/lagrangian_relaxation.html

General assignment problem
Maximize the profit 
finish task within certain budget
each machine can only finish 1 task


Page 341 for GAP definition
task i
machine j
budget B(i)

max  sum[C(i,j)*x(x,j)]
s.t. # within certain budget 
     sum_{j}[A(i,j)*x(i,j)] <= B(i)
     
     # machine can only finish just 1 job
     sum_{i}[x(i,j)] == 1
     x(i,j) in {0,1}
'''

# Solve origimal model

In [38]:
import pyscipopt
from pyscipopt import Model, quicksum, Expr

# ----------------------------------------------------------------------------
# Initialize the problem data
# ----------------------------------------------------------------------------
B = [15, 15, 15]
C = [
    [ 6, 10, 1],
    [12, 12, 5],
    [15,  4, 3],
    [10,  3, 9],
    [8,   9, 5]
]
A = [
    [ 5,  7,  2],
    [14,  8,  7],
    [10,  6, 12],
    [ 8,  4, 15],
    [ 6, 12,  5]
]

# Use CPLEX to double check the model
# ----------------------------------------------------------------------------
# Build the model
# ----------------------------------------------------------------------------
def run_GAP_model_cplex(As, Bs, Cs, **kwargs):
    from docplex.util.environment import get_environment
    from docplex.mp.model import Model
    with Model('GAP per Wolsey -without- Lagrangian Relaxation', **kwargs) as mdl:
        print("#As={}, #Bs={}, #Cs={}".format(len(As), len(Bs), len(Cs)))
        number_of_cs = len(C)
        # variables
        x_vars = [mdl.binary_var_list(c, name=None) for c in Cs]

        # constraints
        mdl.add_constraints(mdl.sum(xv) <= 1 for xv in x_vars) # complicated constraints -- to be removed
        mdl.add_constraints(mdl.sum(x_vars[ii][j] * As[ii][j] for ii in range(number_of_cs)) <= bs
                            for j, bs in enumerate(Bs))

        # objective
        total_profit = mdl.sum(mdl.scal_prod(x_i, c_i) for c_i, x_i in zip(Cs, x_vars))
        mdl.maximize(total_profit)
        
        mdl.print_information()
        # mdl.export_as_lp("/home/tz/Desktop/or/examples/lagrangian_relaxation/test.lp")
    return 0

def run_GAP_model(As, Bs, Cs, **kwargs):
    print("Solve GAP model without RL technique...")
    number_of_cs = len(Cs) # machine i  
    number_of_bs = len(Bs) # task j 
    
    my_toy = Model("origin")
    # variables
    x_vars = [] # 5 * 3
    for i in range(number_of_cs):
        lvship = []
        for j in range(number_of_bs):
            lvship.append(my_toy.addVar(name="x_"+str(i)+"_"+str(j),vtype="B", lb=0.0, ub=1.0))
        x_vars.append(lvship)            

    # constraints
    cons = []
    # 约束1 3 budget constraints
    for j in range(number_of_bs):
        cons.append(my_toy.addCons(quicksum(As[i][j] * x_vars[i][j] for i in range(number_of_cs)) <= Bs[j] ))
                
    # 约束2 5 assignment constraints
    for i in range(number_of_cs):
        cons.append(my_toy.addCons(quicksum(x_vars[i][j] for j in range(number_of_bs)) <= 1.0))
  
    # objective
    my_toy.setObjective(quicksum(Cs[i][j] * x_vars[i][j] \
                                 for i in range(number_of_cs) \
                                 for j in range(number_of_bs)), sense="maximize")
    
    # my_toy.writeProblem('origin.lp')
    my_toy.optimize()
    print("===========================================================================")
    obj = my_toy.getObjVal()
    print("Original model obj is: ", obj)
    for i in range(number_of_cs):
        for j in range(len(Bs)):
            val = my_toy.getVal(x_vars[i][j])     
            if val > 0:                
                print("(%d, %d), profit： %d, budget: %d" % (i, j, Cs[i][j], As[i][j]))
    print("===========================================================================")

    return obj
# To double check the model
# run_GAP_model_cplex(A, B, C)
run_GAP_model(A, B, C)

Solve GAP model without RL technique...
Original model obj is:  46.0
(0, 1), profit： 10, budget: 7
(1, 1), profit： 12, budget: 8
(2, 0), profit： 15, budget: 10
(3, 2), profit： 9, budget: 15
feasible solution found by trivial heuristic after 0.0 seconds, objective value 0.000000e+00
presolving:
(round 1, fast)       0 del vars, 0 del conss, 0 add conss, 0 chg bounds, 2 chg sides, 2 chg coeffs, 0 upgd conss, 0 impls, 15 clqs
   (0.0s) running MILP presolver
   (0.0s) MILP presolver found nothing
(round 2, exhaustive) 0 del vars, 0 del conss, 0 add conss, 0 chg bounds, 2 chg sides, 2 chg coeffs, 8 upgd conss, 0 impls, 15 clqs
(round 3, fast)       0 del vars, 0 del conss, 0 add conss, 0 chg bounds, 3 chg sides, 4 chg coeffs, 8 upgd conss, 0 impls, 15 clqs
(round 4, medium)     0 del vars, 1 del conss, 4 add conss, 0 chg bounds, 7 chg sides, 15 chg coeffs, 8 upgd conss, 0 impls, 15 clqs
   (0.0s) probing cycle finished: starting next cycle
   (0.0s) symmetry computation started: requiring 

46.0

 cliques
presolved problem has 15 variables (15 bin, 0 int, 0 impl, 0 cont) and 11 constraints
      2 constraints of type <knapsack>
      9 constraints of type <setppc>
transformed objective value is always integral (scale: 1)
Presolving Time: 0.00
transformed 1/1 original solutions to the transformed problem space

 time | node  | left  |LP iter|LP it/n|mem/heur|mdpt |vars |cons |rows |cuts |sepa|confs|strbr|  dualbound   | primalbound  |  gap   | compl. 
p 0.0s|     1 |     0 |     0 |     - |  clique|   0 |  15 |  11 |  11 |   0 |  0 |   0 |   0 | 1.120000e+02 | 4.600000e+01 | 143.48%| unknown
  0.0s|     1 |     0 |     3 |     - |   733k |   0 |  15 |   4 |  11 |   0 |  0 |   2 |   0 | 4.600000e+01 | 4.600000e+01 |   0.00%| unknown

SCIP Status        : problem is solved [optimal solution found]
Solving Time (sec) : 0.00
Solving Nodes      : 1
Primal Bound       : +4.60000000000000e+01 (2 solutions)
Dual Bound         : +4.60000000000000e+01
Gap                : 0.00 %


# LR model with SCIP -- variant version

In [9]:
import pyscipopt
from pyscipopt import Model, quicksum, Expr

# ----------------------------------------------------------------------------
# Initialize the problem data
# ----------------------------------------------------------------------------
B = [15, 15, 15]
C = [
    [ 6, 10, 1],
    [12, 12, 5],
    [15,  4, 3],
    [10,  3, 9],
    [8,   9, 5]
]
A = [
    [ 5,  7,  2],
    [14,  8,  7],
    [10,  6, 12],
    [ 8,  4, 15],
    [ 6, 12,  5]
]
def run_GAP_model_with_rl(As, Bs, Cs, **kwargs):
    print("Solve GAP model with RL technique...")
    number_of_cs = len(Cs) # machine i
    number_of_bs = len(Bs) # task j 
          
    my_toy = Model("lr_model")
    
    # variables
    x_vars = [] # 5 * 3    
    p_vars = []
    for i in range(number_of_cs):
        lvship = []
        p_vars.append(my_toy.addVar(name="p_"+str(i),vtype="C", lb=0.0, ub=None))
        for j in range(number_of_bs):
            lvship.append(my_toy.addVar(name="x_"+str(i)+"_"+str(j),vtype="B", lb=0.0, ub=1.0))
        x_vars.append(lvship)            
    
    # constraints
    cons = []
    # 约束1 3 budget constraints
    for j in range(number_of_bs):
        cons.append(my_toy.addCons(quicksum(As[i][j] * x_vars[i][j] for i in range(number_of_cs)) <= Bs[j] ))
                
    # 约束2 5 assignment constraints -- to be penalized
    for i in range(number_of_cs):
        cons.append(my_toy.addCons(quicksum(x_vars[i][j] for j in range(number_of_bs)) <= 1.0 - p_vars[i]))
  
    # objective
    # my_toy.setObjective(quicksum(Cs[i][j] * x_vars[i][j] \
    #                              for i in range(number_of_cs) \
    #                              for j in range(number_of_bs)), sense="maximize")
    
    total_profit = quicksum(Cs[i][j] * x_vars[i][j] \
                                 for i in range(number_of_cs) \
                                 for j in range(number_of_bs))
    
    # Langrangian relaxation loop 
    max_iters = 1
    eps = 1e-6
    loop_count = 0
    best = 0
    initial_multiplier = 1
    multipliers = [initial_multiplier] * len(Cs)
    my_toy.hideOutput(True)
    
    while loop_count <= max_iters:
        loop_count += 1
        total_penalty = quicksum(multipliers[i] * p_vars[i] for i in range(number_of_cs))
        
        my_toy.setObjective(total_profit + total_penalty, sense = "maximize")
        
        my_toy.optimize()
        
        best = my_toy.getObjVal()
        
        penalties = [my_toy.getVal(p_vars[i]) for i in range(number_of_cs)]
        
        print('%d> new lagrangian iteration:\n\t obj=%g, m=%s, p=%s' % (loop_count, best, str(multipliers), str(penalties)))
        do_stop = True
        justifier = 0
        for k in range(number_of_cs):
            penalized_violation = penalties[k] * multipliers[k]
            if penalized_violation >= eps:
                do_stop = False
                justifier = penalized_violation
                break

        if do_stop:
            # print("* Lagrangian relaxation succeeds, best={:g}, penalty={:g}, #iterations={}"
            # .format(best, total_penalty.solution_value, loop_count))
            break
        else:
            # Update multipliers and start the loop again.
            scale_factor = 1.0 / float(loop_count)
            multipliers = [max(multipliers[i] - scale_factor * penalties[i], 0.) for i in range(number_of_cs)]
            print('{0}> -- loop continues, m={1!s}, justifier={2:g}'.format(loop_count, multipliers, justifier)) 
            # my_toy.freeReoptSolve()
            my_toy.freeTransform() # need to free model to re-optimize
    '''
    用SCIP求解的过程与CPLEX有一定的区别(在example-case上)
    # https://dataplatform.cloud.ibm.com/exchange/public/entry/preview?url=https://raw.githubusercontent.com/IBMDataScience/sample-notebooks/master/Cloud/HTML/Use%20Lagrangian%20relaxation.html#Use-Decision-Optimization
    但是LR流程已经跑通
    '''
    # my_toy.writeProblem('origin.lp')
    my_toy.optimize()
    print("===========================================================================")
    obj = my_toy.getObjVal()
    print("Original model obj is: ", obj)
    print("===========================================================================")

    return obj

run_GAP_model_with_rl(A, B, C)

Solve GAP model with RL technique...
1> new lagrangian iteration:
	 obj=47, m=[1, 1, 1, 1, 1], p=[0.0, 0.0, 0.0, 0.0, 1.0]
1> -- loop continues, m=[1.0, 1.0, 1.0, 1.0, 0.0], justifier=1
2> new lagrangian iteration:
	 obj=46, m=[1.0, 1.0, 1.0, 1.0, 0.0], p=[0.0, 0.0, 0.0, 0.0, 0.0]
Original model obj is:  45.99999999999999


45.99999999999999

# Check model with CPLEX

In [42]:
def run_GAP_model_with_Lagrangian_relaxation(As, Bs, Cs, max_iters=1, **kwargs):
    from docplex.util.environment import get_environment
    from docplex.mp.model import Model
    print("Use CPLEX for lagrangian-relaxation...")
    with Model('GAP per Wolsey -with- Lagrangian Relaxation', **kwargs) as mdl:
        print("#As={}, #Bs={}, #Cs={}".format(len(As), len(Bs), len(Cs)))
        number_of_cs = len(Cs)  # =5
        c_range = range(number_of_cs)
        # variables
        x_vars = [mdl.binary_var_list(c, name=None) for c in Cs]
        
        #p_vars are the penalties attached to violating the constraints
        p_vars = mdl.continuous_var_list(Cs, name='p')  # new for relaxation
        
        # new version of the approximated constraint where we apply the penalties
        # sum(xv) <= 1 original constraint
        mdl.add_constraints(mdl.sum(xv) == 1 - pv for xv, pv in zip(x_vars, p_vars))

        mdl.add_constraints(mdl.sum(x_vars[ii][j] * As[ii][j] for ii in c_range) <= bs
                            for j, bs in enumerate(Bs))

        # lagrangian relaxation loop
        eps = 1e-6
        loop_count = 0
        best = 0
        initial_multiplier = 1
        multipliers = [initial_multiplier] * len(Cs)

        total_profit = mdl.sum(mdl.scal_prod(x_i, c_i) for c_i, x_i in zip(Cs, x_vars))
        mdl.add_kpi(total_profit, "Total profit")

        while loop_count <= max_iters:
            loop_count += 1
            # rebuilt at each loop iteration
            total_penalty = mdl.scal_prod(p_vars, multipliers)
            mdl.maximize(total_profit + total_penalty)
            # mdl.print_information()
            mdl.export_as_lp("/home/tz/Desktop/or/examples/lagrangian_relaxation/rl_"+str(loop_count)+".lp")
            # s = mdl.solve()
            # if not s:
            #    print("*** solve fails, stopping at iteration: %d" % loop_count)
            #     break
            
            # best = s.objective_value
            # penalties = [pv.solution_value for pv in p_vars]
            penalties = [0, 0, 0, 0, 1]
            # print('%d> new lagrangian iteration:\n\t obj=%g, m=%s, p=%s' % (loop_count, best, str(multipliers), str(penalties)))

            do_stop = True
            justifier = 0
            for k in c_range:
                penalized_violation = penalties[k] * multipliers[k]
                if penalized_violation >= eps:
                    do_stop = False
                    justifier = penalized_violation
                    break

            if do_stop:
                # print("* Lagrangian relaxation succeeds, best={:g}, penalty={:g}, #iterations={}"
                #       .format(best, total_penalty.solution_value, loop_count))
                break
            else:
                # update multipliers and start loop again.
                scale_factor = 1.0 / float(loop_count)
                multipliers = [max(multipliers[i] - scale_factor * penalties[i], 0.) for i in c_range]
                print('{0}> -- loop continues, m={1!s}, justifier={2:g}'.format(loop_count, multipliers, justifier))

    return 0
run_GAP_model_with_Lagrangian_relaxation(A, B, C)

#As=5, #Bs=3, #Cs=5
1> -- loop continues, m=[1.0, 1.0, 1.0, 1.0, 0.0], justifier=1


0

# Standard LR framework -- subgradient method

## Using $\frac{1}{K}$ to update step size

In [41]:
import pyscipopt
from pyscipopt import Model, quicksum, Expr

# ----------------------------------------------------------------------------
# Initialize the problem data
# ----------------------------------------------------------------------------
B = [15, 15, 15]
C = [
    [ 6, 10, 1],
    [12, 12, 5],
    [15,  4, 3],
    [10,  3, 9],
    [8,   9, 5]
]
A = [
    [ 5,  7,  2],
    [14,  8,  7],
    [10,  6, 12],
    [ 8,  4, 15],
    [ 6, 12,  5]
]
def run_GAP_model_with_rl(As, Bs, Cs, **kwargs):
    print("Solve GAP model with RL technique...")
    number_of_cs = len(Cs) # machine i |i|=5
    number_of_bs = len(Bs) # task j |j|=3
    
    # =========================================================================================     
    my_toy = Model("lr_model")
    
    # variables
    x_vars = [] # 5 * 3    
    for i in range(number_of_cs):
        lvship = []
        for j in range(number_of_bs):
            lvship.append(my_toy.addVar(name="x_"+str(i)+"_"+str(j),vtype="B", lb=0.0, ub=1.0))
        x_vars.append(lvship)            
    
    # constraints
    cons = []
    # 约束1 3 budget constraints
    for j in range(number_of_bs):
        cons.append(my_toy.addCons(quicksum(As[i][j] * x_vars[i][j] for i in range(number_of_cs)) <= Bs[j] ))
                    
    total_profit = quicksum(Cs[i][j] * x_vars[i][j] \
                                 for i in range(number_of_cs) \
                                 for j in range(number_of_bs))
    # =========================================================================================     
    # Langrangian relaxation loop 
    max_iters = 100
    eps = 1e-6
    loop_count = 0
    best = 0

    initial_multiplier = 1.0
    multipliers = [initial_multiplier] * len(Cs)
    my_toy.hideOutput(True)
    
    while loop_count <= max_iters:
        loop_count += 1
        total_penalty = quicksum(multipliers[i] * quicksum(1.0 - x_vars[i][j] for j in range(number_of_bs)) 
                                 for i in range(number_of_cs))
        
        my_toy.setObjective(total_profit + total_penalty, sense = "maximize")

        print("Solving LR sub-prob...")
        my_toy.optimize()
        
        best = my_toy.getObjVal()
        
        x_bar = []
        sub_gradient = []
        original_obj = 0
        for i in range(number_of_cs):
            temp = []
            delta = 0
            for j in range(number_of_bs):
                x_sol = my_toy.getVal(x_vars[i][j])
                delta = delta - x_sol
                temp.append(x_sol)
                original_obj += Cs[i][j] * x_sol
                
            delta = 1 + delta
            sub_gradient.append(delta)    
            x_bar.append(temp)      
        
        # Calculate sub-gradient: Page 336
        # step-size = 1/k 算法收敛速度非常慢 不适用
        step_size = 1.0 / float(loop_count) # can be replaced by step-size function 
        old_multipliers = multipliers
        multipliers = [max(0.0, multipliers[i]-step_size*sub_gradient[i]) for i in range(len(multipliers))]
        print("===========================================================================")
        print("Original-prob obj: %d, RL obj: %4.5f" %(original_obj, best))
        
        do_stop = True
        justifier = 0

        # 计算2次 sub-gradient的 L2-norm -- 是否停止迭代        
        for k in range(number_of_cs):
            penalized_violation = pow(old_multipliers[k] - multipliers[k], 2)
            if penalized_violation >= eps:
                do_stop = False
                justifier = penalized_violation
                break

        if do_stop:
            print("* Lagrangian relaxation succeeds, best={:g}, #iterations={}".format(best, loop_count))
            break
        else:
            # Update multipliers and start the loop again.            
            # print('{0}> --> justifier={1:g}'.format(loop_count, justifier)) 
            my_toy.freeTransform() # need to free model to re-optimize

    # my_toy.writeProblem('origin.lp')
    print("===========================================================================")
    print("Model obj is: ", best)
    original_obj = 0
    for i in range(number_of_cs):
        for j in range(number_of_bs):
            original_obj += Cs[i][j] * x_bar[i][j] 
            if x_bar[i][j] > 0:
                print("(%d, %d), profit： %d, budget: %d" % (i, j, Cs[i][j], As[i][j]))
    print("The original problem obj value is: ", original_obj)
    print("===========================================================================")

    return original_obj

run_GAP_model_with_rl(A, B, C)

Solve GAP model with RL technique...
Solving LR sub-prob...
Original-prob obj: 52, RL obj: 62.00000
Solving LR sub-prob...
Original-prob obj: 53, RL obj: 61.00000
Solving LR sub-prob...
Original-prob obj: 53, RL obj: 60.50000
Solving LR sub-prob...
Original-prob obj: 49, RL obj: 61.50000
Solving LR sub-prob...
Original-prob obj: 49, RL obj: 61.00000
Solving LR sub-prob...
Original-prob obj: 52, RL obj: 60.83333
Solving LR sub-prob...
Original-prob obj: 52, RL obj: 61.00000
Solving LR sub-prob...
Original-prob obj: 49, RL obj: 61.21905
Solving LR sub-prob...
Original-prob obj: 52, RL obj: 61.14286
Solving LR sub-prob...
Original-prob obj: 52, RL obj: 61.25397
Solving LR sub-prob...
Original-prob obj: 49, RL obj: 61.39127
Solving LR sub-prob...
Original-prob obj: 52, RL obj: 61.35397
Solving LR sub-prob...
Original-prob obj: 52, RL obj: 61.43730
Solving LR sub-prob...
Original-prob obj: 49, RL obj: 61.52996
Solving LR sub-prob...
Original-prob obj: 53, RL obj: 61.58490
Solving LR sub-pro

49.0

## Using an another method to update step-size

In [125]:
import pyscipopt
from pyscipopt import Model, quicksum, Expr
import copy as cp
# ----------------------------------------------------------------------------
# Initialize the problem data
# This is a maximization problem -- different from the minimization problem
# >>> the LB and UB should be treated carefully
# ----------------------------------------------------------------------------
B = [15, 15, 15]
C = [
    [ 6, 10, 1],
    [12, 12, 5],
    [15,  4, 3],
    [10,  3, 9],
    [8,   9, 5]
]
A = [
    [ 5,  7,  2],
    [14,  8,  7],
    [10,  6, 12],
    [ 8,  4, 15],
    [ 6, 12,  5]
]


'''
The step size is updated using LB since in this problem the UB is hard to obtain 
Compared with another example in https://scipopt.github.io/PySCIPOpt/docs/html/classpyscipopt_1_1scip_1_1Model.html#a59ac011e2fcca0dec49e8da06443ba90
whose UB is easy to obtain

The step size is updated by following formula
mu = theta * [UB - obj] / ||sub-gradient||

[UB - obj] is the crutial part
if the original problem is Minimization problem this part is: [UB - obj] 
>>> the Lagrangian dual is max min xxx --> UB is from the outter prob --> the difference between UB and current sol
>>> when obj ~ UB -> solution obtained
>>> step size is determined by outter problem: UB vs Obj

>>> the Lagrangian dual is min max xxx --> LB is from the outter prob --> 
>>> in this case LB is the customized LB -> when obj ~ LB -> solution obtained  
>>> step size is determined by outter problem: LB vs Obj
'''
def run_GAP_model_with_rl(As, Bs, Cs, **kwargs):
    print("Solve GAP model with RL technique...")
    number_of_cs = len(Cs) # machine i |i|=5
    number_of_bs = len(Bs) # task j |j|=3
    
    steplog = []
    scalelog = []
    LBlog = []
    UBlog = []
    # =========================================================================================     
    my_toy = Model("lr_model")
    
    # variables
    x_vars = [] # 5 * 3    
    for i in range(number_of_cs):
        lvship = []
        for j in range(number_of_bs):
            lvship.append(my_toy.addVar(name="x_"+str(i)+"_"+str(j),vtype="B", lb=0.0, ub=1.0))
        x_vars.append(lvship)            
    
    # constraints
    cons = []
    # 约束1 3 budget constraints
    for j in range(number_of_bs):
        cons.append(my_toy.addCons(quicksum(As[i][j] * x_vars[i][j] for i in range(number_of_cs)) <= Bs[j] ))
                    
    total_profit = quicksum(Cs[i][j] * x_vars[i][j] \
                                 for i in range(number_of_cs) \
                                 for j in range(number_of_bs))
    # =========================================================================================     
    # Langrangian relaxation loop 
    max_iters = 100
    eps = 1e-6
    loop_count = 0
    best = 0

    initial_multiplier = 0.0
    multipliers = [initial_multiplier] * len(Cs)
    my_toy.hideOutput(True)
    
    same = 0
    norm = 0.0
    step_size = 0.0
    scale = 1.0
    
    # Initialize UB, LB
    UB = 0
    for i in range(number_of_cs):
        UB += max(Cs[i][:])
        
    LB = 0
    for j in range(number_of_bs):
        LB += max(Cs[:][j])            
    
    while loop_count <= max_iters:
        loop_count += 1
        total_penalty = quicksum(multipliers[i] * quicksum(1.0 - x_vars[i][j] for j in range(number_of_bs)) 
                                 for i in range(number_of_cs))
        
        my_toy.setObjective(total_profit + total_penalty, sense = "maximize")

        my_toy.optimize()
        
        obj = my_toy.getObjVal()
        
        # Update step size >>> upper bound is the relaxed problem solution
        if my_toy.getObjVal() < LB + 1e-6:
            same = 0
        else:
            same += 1
        # update parameter theta
        if same >= 3:
            scale /= 2.0
            same = 0
            
        x_bar = []
        sub_gradient = []
        original_obj = 0
        for i in range(number_of_cs):
            temp = []
            delta = 0
            for j in range(number_of_bs):
                x_sol = my_toy.getVal(x_vars[i][j])
                delta = delta - x_sol
                temp.append(x_sol)
                original_obj += Cs[i][j] * x_sol
                
            delta = 1 + delta
            sub_gradient.append(delta)    
            x_bar.append(temp)      
        
        norm = sum(sub_gradient[i]**2.0 for i in range(len(sub_gradient)))
        
        step_size = scale * (obj-LB) / norm
        
        # Calculate sub-gradient: Page 336
        # step_size = 1.0 / float(loop_count) # step-size = 1/k 算法收敛速度非常慢 不适用 
        old_multipliers = cp.deepcopy(multipliers)
        for i in range(len(multipliers)):
            if multipliers[i] > step_size*sub_gradient[i]:
                multipliers[i] -= step_size*sub_gradient[i]
            else:
                multipliers[i] = 0.0
        
        # ===========================================================================
        # Update UB, LB
        UB = my_toy.getObjVal()
        eps = sum((multipliers[i] - old_multipliers[i])**2 for i in range(len(multipliers)))

        temp = 0.0
        for i in range(number_of_cs):
            for j in range(number_of_bs):
                if x_bar[i][j] > 0:
                    temp += Cs[i][j] * x_bar[i][j] 
                    break
                    
        LB = max(LB, temp)
        
        LBlog.append(LB)
        UBlog.append(UB)
        steplog.append(step_size)
        scalelog.append(scale)
        
        my_toy.freeTransform() # need to free model to re-optimize
        if step_size < 1e-6 or eps < 1e-6:
            break
            
    # my_toy.writeProblem('origin.lp')
    print("===========================================================================")
    print("\n               *** Summary Report ***               \n")
    print("  Iter        LB               UB          scale        step")
    for i in range(len(LBlog)):
            print("  %3d    %12.4f    %12.4f    %8.4f    %8.4f" \
                  % (i, LBlog[i], UBlog[i], scalelog[i], steplog[i]))
    
    original_obj = 0
    for i in range(number_of_cs):
        for j in range(number_of_bs):
            original_obj += Cs[i][j] * x_bar[i][j] 
            if x_bar[i][j] > 0:
                print("(%d, %d), profit： %d, budget: %d" % (i, j, Cs[i][j], As[i][j]))
    print("===========================================================================")

    return original_obj

run_GAP_model_with_rl(A, B, C)

Solve GAP model with RL technique...

               *** Summary Report ***               

  Iter        LB               UB          scale        step
    0         38.0000         54.0000      1.0000      2.8333
    1         40.0000         66.0000      1.0000     14.0000
    2         40.0000        103.5000      0.5000     10.5833
    3         40.0000        128.8333      0.5000     22.2083
    4         40.0000         64.2500      0.5000      2.0208
    5         40.0000         63.2500      0.2500      1.9375
    6         40.0000         68.8750      0.2500      3.6094
    7         40.0000         73.7656      0.2500      2.8138
    8         40.0000         83.7187      0.1250      2.7324
    9         40.0000         78.7819      0.1250      1.6159
   10         42.0000         80.2767      0.1250      2.5173
   11         42.0000         85.5779      0.0625      0.5447
   12         46.0000         83.2112      0.0625      2.5757
   13         46.0000         81.6498    

46.0